# 03: LSTM Text Generation

In [65]:
import numpy as np
from pickle import load
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

np.random.seed(0)

In [66]:
def load_sequences(path_and_filename):
    sequence_data = open(path_and_filename).read()
    sequences = sequence_data.split('\n')
    
    words_in_seq = len(sequences[0].split()) - 1
    
    print(f'{len(sequences)} sequences have been loaded.')
    print(f'Each sequence has {words_in_seq} word token(s) plus an output token.')
    return sequences, words_in_seq

In [67]:
sequence_list, seq_length = \
load_sequences('../data/Poe_NLG/03_Text_files_for_models/cleaned_poe_tot_seq_len_26.txt')

480044 sequences have been loaded.
Each sequence has 25 word token(s) plus an output token.


In [68]:
model = load_model('./Models/Models_25_seqlen_LSTM_model/25_seqlen_LSTM_model_word_model.h5')

In [69]:
tokenizer = load(open('./Models/Models_25_seqlen_LSTM_model/25_seqlen_LSTM_model_tokenizer.pkl', 'rb'))

In [70]:
seed_text = sequence_list[np.random.randint(0, len(sequence_list))]

In [71]:
print(seed_text + '\n')

the most remote degree , connected with my family . But assuredly if we had been brothers we must have been twins ; for , after



In [72]:
def generate_text(model, tokenizer, seq_length, seed_text, num_new_words):
    result = []
    input_text = seed_text
    for _ in range(num_new_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        predictions = model.predict(encoded)
#         predictions = (model.predict(encoded) > 0.5).astype("int32")
        pred = np.random.choice(len(predictions[0]), p=predictions[0])
        # model.predict_classes(encoded, verbose=0)
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == pred:
                output_word = word
                break
        input_text += ' ' + output_word
        result.append(output_word)
    return ' '.join(result)

In [73]:
new_stuff = generate_text(model, tokenizer, seq_length, seed_text, 50)
print(new_stuff)

others further north north fall another five my room going numerous going mere total certain personal human birth what knowing evident frequent none giving occasion others eight fifteen its evident every anything those their certain them those Augustus cutting late lying cutting its cover sail other several drawing accident certain


In [74]:
print(seed_text + '\n')
print(new_stuff)

the most remote degree , connected with my family . But assuredly if we had been brothers we must have been twins ; for , after

others further north north fall another five my room going numerous going mere total certain personal human birth what knowing evident frequent none giving occasion others eight fifteen its evident every anything those their certain them those Augustus cutting late lying cutting its cover sail other several drawing accident certain
